In [8]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# https://googlechromelabs.github.io/chrome-for-testing/
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from dotenv import load_dotenv
import time
from IPython.display import clear_output

Download the latest Chrome driver version:

https://googlechromelabs.github.io/chrome-for-testing/

### Open website and accept cookies

In [9]:
year = 2025
user_name = "DavidG85"
load_dotenv("../../apis/.env")
chromedriver_path = os.getenv("CHROMEDRIVER_PATH")

service = Service(executable_path=chromedriver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.get(f'https://www.last.fm/user/{user_name}/library/artists?from={year}-01-01&to={year}-12-31&page=1')
# time.sleep(2)
accept_cookies_button = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
print(accept_cookies_button.text)
accept_cookies_button.click()

Accept All


In [62]:
# driver.close()

### Login

In [10]:
lfm_pwd = os.getenv("LFM")
username_box = driver.find_element(By.XPATH, '//*[@id="id_username_or_email"]')
pwd_box = driver.find_element(By.XPATH, '//*[@id="id_password"]')
username_box.send_keys(user_name)
pwd_box.send_keys(lfm_pwd)
print("User: ",user_name)
print("Password: ","*"*len(lfm_pwd))
signin_button = driver.find_element(By.XPATH, '//*[@id="login"]/div[3]/div/button')
print(signin_button.text)
signin_button.click()

User:  DavidG85
Password:  ***************
LET ME IN!


### Extract HTML data with Selenium

Extracting a month:

In [ ]:
from calendar import monthrange

def retrieve_ranking(start_date, end_date, n_pages, driver, user_name = 'DavidG85'):
    html_sources = []
    for page in range(n_pages):
        driver.implicitly_wait(10)
        driver.get(f'https://www.last.fm/user/{user_name}/library/artists?from={start_date}&to={end_date}&page={page+1}')
        html_source = driver.page_source
        clear_output(wait=True)
        print(f"Page {page+1} of {n_pages}")
        print(html_source[:1000],"\n...")
        html_sources.append(html_source)
    return html_sources

def get_month_date_range(year, month):
    start_date = f"{year}-{month:02d}-01"
    end_day = monthrange(year, month)[1]
    end_date = f"{year}-{month:02d}-{end_day}"
    return start_date, end_date

year = 2025
month = 4
start_date, end_date = get_month_date_range(year, month)
override_dates = False # True for seasons
if override_dates:
    start_date = '2024-12-21'
    end_date = '2025-03-20'
print(f"Start Date: {start_date}, End Date: {end_date}")
n_pages = 1

html_sources = retrieve_ranking(start_date, end_date, n_pages, driver, user_name)

Page 1 of 1
<html lang="en" class="
        js
        playbar-masthead-release-shim
        youtube-provider-not-ready
     video svgforeignobject csspointerevents"><head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript" src="//zn394yygoj0zd35dd-cbs.siteintercept.qualtrics.com/WRSiteInterceptEngine/?Q_ZID=ZN_394YyGOJ0Zd35DD&amp;Q_LOC=https%3A%2F%2Fwww.last.fm%2Fuser%2FDavidG85%2Flibrary%2Fartists%3Ffrom%3D2024-12-21%26to%3D2025-03-20%26page%3D1"></script><script type="text/javascript" async="" id="tealium-tag-3005" src="https://sb.scorecardresearch.com/c2/3005086/cs.js"></script><script async="" src="https://www.everestjs.net/static/le/last-event-tag-latest.min.js"></script><script type="text/javascript" id="googletag-script" src="//securepubads.g.doubleclick.net/tag/js/gpt.js" async=""></script><script async="" src="https://strangeclocks.com/v2fjjH2X7fDDVyjq3kQFUt12y5wXOkb_ai99HR2WWrKQDN2VxaMxDPn0U"></script><s

Extracting a year:

In [9]:
start_date = '2024-01-01'
end_date = '2024-12-31'
n_pages = 10

html_sources = retrieve_ranking(start_date, end_date, n_pages, driver)

Page 10 of 10
<html lang="en" class="
        js
        playbar-masthead-release-shim
        youtube-provider-not-ready
     video svgforeignobject csspointerevents"><head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript" id="bb-liveconnect" src="//b-code.liadm.com/b-01rk.min.js" async="" charset="utf-8"></script><script type="text/javascript" id="googletag-script" src="//securepubads.g.doubleclick.net/tag/js/gpt.js" async=""></script><script async="" src="https://strangeclocks.com/v2fjjH2X7fDDVyjq3kQFUt12y5wXOkb_ai99HR2WWrKQDN2VxaMxDPn0U"></script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"0ed0ce50b0","applicationID":"5588594","transactionName":"Y1BRbUUFWkMEWxYMV1oadUxZB0BZClZNCV5ZG0RcVUpDRxIWAxVIRxtGSlIWGkYMXRUWFlhcUUtWFk0KJEoWDEtARh1eUhA=","queueTime":0,"applicationTime":680,"agent":""};
(window.NREUM||(NREUM={}

### Parse with BeautifulSoup and get rankings

In [12]:
df_bands = pd.DataFrame(columns=['orig_rank','rank','Artist', 'Orig_points', 'extra'])

# Loop through each HTML element in the list
for html_source in html_sources:
    soup = BeautifulSoup(html_source, 'html.parser')
    meta_tags = soup.find_all('meta', property='og:description')
    
    if len(meta_tags) > 1:
        content = meta_tags[1]['content']
        bands_data = re.findall(r'([\w\s&.-]+) \((\d+)\)', content)
        
        # Create a DataFrame for the current HTML source
        df_temp = pd.DataFrame(bands_data, columns=['Artist', 'Orig_points'])
        df_temp['Orig_points'] = pd.to_numeric(df_temp['Orig_points'])
        
        # Append the current DataFrame to the main DataFrame
        df_bands = pd.concat([df_bands, df_temp], ignore_index=True)


df_bands.index+=1
df_bands.orig_rank = df_bands.index
# df_bands.rank = '=ROW(A2)-1'
df_bands['rank'] = df_bands['orig_rank'].apply(lambda x: f'=ROW(A{x+1})-1')
df_bands['total'] = df_bands['orig_rank'].apply(lambda x: f'=SUM(D{x+1}:E{x+1})')
df_bands['points%'] = df_bands['orig_rank'].apply(lambda x: f'=F{x+1}/$O$2')
df_bands['points'] = df_bands['orig_rank'].apply(lambda x: f'=F{x+1}*$N$2/$O$2')
df_bands['ROUND'] = df_bands['orig_rank'].apply(lambda x: f'=ROUND(H{x+1},0)')
df_bands['Acum'] = df_bands['orig_rank'].apply(lambda x: f'=IFERROR(J{x}+I{x+1},I{x+1})')
df_bands['PUNTOS'] = df_bands['orig_rank'].apply(lambda x: f'=IF(J{x+1}>365,0,I{x+1})')
df_bands['COUNTRY'] = ''
df_bands['PAD'] = ''
df_bands['days_yr'] = '' 
df_bands.loc[1, 'days_yr'] = 365
df_bands['puntos_top_170'] = '' 
df_bands.loc[1, 'puntos_top_170'] = '=SUM(F2:F171)'
df_bands['Puntos_finales_170'] = '' 
df_bands.loc[1, 'Puntos_finales_170'] = '=SUM(I2:I171)'
df_bands['avg_artist_points'] = '' 
df_bands.loc[1, 'avg_artist_points'] = '=N2/50'

display(df_bands)
df_bands.to_csv(f'ranking_{start_date}_{end_date}.csv', index=False)
temp_path = 'C:/Users/david/My Drive/Music/Music documents/csv'
df_bands.to_csv(f'{temp_path}/ranking_{start_date}_{end_date}.csv', index=False)

,orig_rank,rank,Artist,Orig_points,extra,total,points%,points,ROUND,Acum,PUNTOS,COUNTRY,PAD,days_yr,puntos_top_170,Puntos_finales_170,avg_artist_points
1,1,=ROW(A2)-1,Kanye West,85,NaN,=SUM(D2:E2),=F2/$O$2,=F2*$N$2/$O$2,"=ROUND(H2,0)","=IFERROR(J1+I2,I2)","=IF(J2>365,0,I2)",,,365,=SUM(F2:F171),=SUM(I2:I171),=N2/50
2,2,=ROW(A3)-1,Los Planetas,71,NaN,=SUM(D3:E3),=F3/$O$2,=F3*$N$2/$O$2,"=ROUND(H3,0)","=IFERROR(J2+I3,I3)","=IF(J3>365,0,I3)",,,,,,
3,3,=ROW(A4)-1,Arde Bogotá,70,NaN,=SUM(D4:E4),=F4/$O$2,=F4*$N$2/$O$2,"=ROUND(H4,0)","=IFERROR(J3+I4,I4)","=IF(J4>365,0,I4)",,,,,,
4,4,=ROW(A5)-1,Caroline Polachek,68,NaN,=SUM(D5:E5),=F5/$O$2,=F5*$N$2/$O$2,"=ROUND(H5,0)","=IFERROR(J4+I5,I5)","=IF(J5>365,0,I5)",,,,,,
5,5,=ROW(A6)-1,Eugenio,66,NaN,=SUM(D6:E6),=F6/$O$2,=F6*$N$2/$O$2,"=ROUND(H6,0)","=IFERROR(J5+I6,I6)","=IF(J6>365,0,I6)",,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,492,=ROW(A493)-1,Arlo Parks,2,NaN,=SUM(D493:E493),=F493/$O$2,=F493*$N$2/$O$2,"=ROUND(H493,0)","=IFERROR(J492+I493,I493)","=IF(J493>365,0,I493)",,,,,,
493,493,=ROW(A494)-1,Aryana León,2,NaN,=SUM(D494:E494),=F494/$O$2,=F494*$N$2/$O$2,"=ROUND(H494,0)","=IFERROR(J493+I494,I494)","=IF(J494>365,0,I494)",,,,,,
494,494,=ROW(A495)-1,Aztec Camera,2,NaN,=SUM(D495:E495),=F495/$O$2,=F495*$N$2/$O$2,"=ROUND(H495,0)","=IFERROR(J494+I495,I495)","=IF(J495>365,0,I495)",,,,,,
495,495,=ROW(A496)-1,Bad Omens,2,NaN,=SUM(D496:E496),=F496/$O$2,=F496*$N$2/$O$2,"=ROUND(H496,0)","=IFERROR(J495+I496,I496)","=IF(J496>365,0,I496)",,,,,,


In [8]:
driver.close()

## Use cases

See below some screen captures from the spreadsheet where I've been tracking and curating my music listening stats (from last.fm and other non-automated sources) for more than 20 years

![](imgs/img1.png)  
![](imgs/img2.png)  
![](imgs/img3.png)  
![](imgs/img4.png)  
![](imgs/img5.png)